In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pycaret
from pycaret.datasets import get_data
dataset= pd.read_csv('manufacturing dataset.csv')

In [3]:
dataset.head()

,Time,0,1,2,3,4,5,6,7,8,...,581,582,583,584,585,586,587,588,589,Pass/Fail
0,2008-07-19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,...,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,2008-07-19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2008-07-19 13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2008-07-19 14:43:00,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,...,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1
4,2008-07-19 15:22:00,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,...,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1


In [4]:
dataset.isnull().sum()

Time          0
0             6
1             7
2            14
3            14
             ..
586           1
587           1
588           1
589           1
Pass/Fail     0
Length: 592, dtype: int64

In [5]:
dataset.columns

Index(['Time', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '581', '582', '583', '584', '585', '586', '587', '588', '589',
       'Pass/Fail'],
      dtype='object', length=592)

In [6]:
dataset.columns = 'features_'+dataset.columns

In [7]:
dataset.rename(columns = {'features_Time': 'Time'}, inplace=True)
dataset.rename(columns = {'features_Pass/Fail': 'Pass_Fail'}, inplace=True)

In [8]:
df = dataset.iloc[:,1:]
df = df.apply(lambda x:x.fillna(0), axis=0)
# axis=0 - row wise
df2 = dataset.iloc[:,0]
result = pd.concat([df, df2], axis=1).reindex(df.index)
# axis=1

In [9]:
result.head()

,features_0,features_1,features_2,features_3,features_4,features_5,features_6,features_7,features_8,features_9,...,features_582,features_583,features_584,features_585,features_586,features_587,features_588,features_589,Pass_Fail,Time
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,...,0.5005,0.0118,0.0035,2.3630,0.0000,0.0000,0.0000,0.0000,-1,2008-07-19 11:55:00
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,...,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1,2008-07-19 12:32:00
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,...,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1,2008-07-19 13:17:00
3,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,-0.0124,...,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1,2008-07-19 14:43:00
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,-0.0031,...,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1,2008-07-19 15:22:00


In [10]:
from datetime import datetime
result['year'] = pd.DatetimeIndex(result['Time']).year
result['month'] = pd.DatetimeIndex(result['Time']).month
result['date'] = pd.DatetimeIndex(result['Time']).day
result['week_day'] = pd.DatetimeIndex(result['Time']).weekday
result['start_time'] = pd.DatetimeIndex(result['Time']).time
result['hour'] = pd.DatetimeIndex(result['Time']).hour
result['min'] = pd.DatetimeIndex(result['Time']).minute

In [11]:
# result.head()
# Now we should divide out input features and target features 
x= result.drop(['Pass_Fail','Time','year','start_time'], axis=1)
y=result['Pass_Fail']

In [12]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [13]:
x_scaler= scaler.fit_transform(x)

In [14]:
x=pd.DataFrame(x_scaler,columns=x.columns[:])

In [15]:
y.value_counts()

-1    1463
 1     104
Name: Pass_Fail, dtype: int64

In [16]:
import imblearn
from imblearn.over_sampling import RandomOverSampler

In [17]:
ros=RandomOverSampler()
x_ros,y_ros=ros.fit_resample(x,y)
from collections import Counter
print(Counter(y))
print("*****"*5)
print(Counter(y_ros))

Counter({-1: 1463, 1: 104})
*************************
Counter({-1: 1463, 1: 1463})


In [18]:
# splitting the data into training and test
# training data
data = result.sample(frac=0.85, random_state=1).reset_index(drop=True)
# test data
data_unseen = result.drop(data.index).reset_index(drop=True)

In [19]:
data.head()

,features_0,features_1,features_2,features_3,features_4,features_5,features_6,features_7,features_8,features_9,...,features_589,Pass_Fail,Time,year,month,date,week_day,start_time,hour,min
0,3042.08,2694.31,2206.5112,1244.1552,1.2691,100.0,101.6667,0.1229,1.5928,0.0163,...,107.8331,-1,2008-07-31 15:36:00,2008,7,31,3,15:36:00,15,36
1,2982.40,2528.21,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,1.4805,0.0123,...,102.5241,-1,2008-09-21 18:12:00,2008,9,21,6,18:12:00,18,12
2,2898.17,2547.65,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,1.4425,0.0222,...,185.3578,-1,2008-12-10 03:21:00,2008,12,10,2,03:21:00,3,21
3,2956.77,2518.50,2187.4111,1942.3069,1.1864,100.0,88.0911,0.1245,1.4113,0.0108,...,20.3091,-1,2008-09-19 01:57:00,2008,9,19,4,01:57:00,1,57
4,2936.59,2526.22,2196.6889,1593.1220,1.5925,100.0,99.1133,0.1226,1.4934,-0.0074,...,56.0858,1,2008-08-20 02:27:00,2008,8,20,2,02:27:00,2,27


In [20]:
x= result.drop('Pass_Fail', axis=1)
y=result['Pass_Fail']

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.85)

In [22]:
print("Data for Modeling :" + str(data.shape))
print("###########"*10)
print("Unseen Data for Prediction :" + str(data_unseen.shape))

Data for Modeling :(1332, 599)
##############################################################################################################
Unseen Data for Prediction :(235, 599)


In [23]:
from pycaret.classification import *

In [24]:
# exp_clf101 = setup(data=data, target='Pass_Fail', fold_shuffle=True, session_id=123, imputation_type='iterative')

In [25]:
exp_clf101 = setup(data,session_id = 123,target = 'Pass_Fail',fold_shuffle=True, imputation_type='iterative',multicollinearity_threshold = 0.95,)

,Description,Value
0,session_id,123
1,Target,Pass_Fail
2,Target Type,Binary
3,Label Encoded,"-1: 0, 1: 1"
4,Original Data,"(1332, 599)"
5,Missing Values,False
6,Numeric Features,589
7,Categorical Features,8
8,Ordinal Features,False
9,High Cardinality Features,False


AttributeError: 'Make_Time_Features' object has no attribute 'list_of_features'

In [ ]:
compare_models()

IntProgress(value=0, description='Processing: ', max=79)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,04:17:19
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
